In [7]:
import pandas as pd
from IPython.display import display
# Path to the CSV file
csv_file_path = 'slot_1_updated_cs1.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(csv_file_path)

# Create the initial dataframe
df = pd.DataFrame(df)

emergency_left_vehicles = pd.DataFrame()
# Display the first few rows of the DataFrame to verify
df

,Car Number,SOC Initial,Preference Slots,Route,Priority,Battery Capacity,Full Charge Range,KM before CS,Required time(If user provide),SOC Final,Required Time (Calculated)
0,23123,35,1,1,0,90.0,350.0,25.0,15.0,30,16
1,24,35,1,1,0,NaN,NaN,NaN,17.0,31,15
2,45345,28,1,1,1,NaN,NaN,NaN,15.0,22,13
3,76786,65,1,1,1,NaN,NaN,NaN,NaN,60,18
4,5677,28,1,2,2,NaN,NaN,NaN,15.0,24,17
5,768678,44,1,1,2,NaN,NaN,NaN,NaN,40,14
6,75675,14,1,2,3,NaN,NaN,NaN,NaN,10,16
7,452,16,1,3,3,NaN,NaN,NaN,NaN,12,16
8,5645,18,1,3,3,NaN,NaN,NaN,NaN,15,14
9,7967,31,1,3,3,NaN,NaN,NaN,NaN,27,13


In [8]:
# Calculate the cumulative sum of required_time and add it as a new column final_cumulative_time
df['cumulative_time'] = df['Required Time (Calculated)'].cumsum()
df

,Car Number,SOC Initial,Preference Slots,Route,Priority,Battery Capacity,Full Charge Range,KM before CS,Required time(If user provide),SOC Final,Required Time (Calculated),cumulative_time
0,23123,35,1,1,0,90.0,350.0,25.0,15.0,30,16,16
1,24,35,1,1,0,NaN,NaN,NaN,17.0,31,15,31
2,45345,28,1,1,1,NaN,NaN,NaN,15.0,22,13,44
3,76786,65,1,1,1,NaN,NaN,NaN,NaN,60,18,62
4,5677,28,1,2,2,NaN,NaN,NaN,15.0,24,17,79
5,768678,44,1,1,2,NaN,NaN,NaN,NaN,40,14,93
6,75675,14,1,2,3,NaN,NaN,NaN,NaN,10,16,109
7,452,16,1,3,3,NaN,NaN,NaN,NaN,12,16,125
8,5645,18,1,3,3,NaN,NaN,NaN,NaN,15,14,139
9,7967,31,1,3,3,NaN,NaN,NaN,NaN,27,13,152


In [9]:
# Define the value of X
X = int(input("When will the emergency vehicle enter? "))
Y = int(input("Required Charging time for emergency vehicle?"))

# Find the index of the first row where cumulative_time is greater than or equal to X
split_index = df[df['cumulative_time'] >= X].index[0]

# Split the dataframe into two parts
df1 = df.iloc[:split_index + 1]
df2 = df.iloc[split_index + 1:]

# Duplicate the last row of df1 and add it to the end of df1
last_row = df1.iloc[[-1]]
df1 = pd.concat([df1, last_row], ignore_index=True)

# Perform the operation: Minus the value of X from the cumulative_time of the last row and put the value to the required_time of that row
df1.at[df1.index[-2], 'Required Time (Calculated)'] = X - df1.at[df1.index[-3], 'cumulative_time']

# Perform the operation: Minus the value of X from the cumulative_time of the last row and put the value to the required_time of that row
df1.at[df1.index[-1], 'Required Time (Calculated)'] = df1.at[df1.index[-1], 'Required Time (Calculated)'] - df1.at[df1.index[-2], 'Required Time (Calculated)']

# Create a new row with required_time = 10 and other columns as null
new_row = pd.DataFrame({'Car Number': ['0000'],'SOC Initial': [None],'Preference Slots': [None],'Route': [None],'Priority': [None],'Battery Capacity': [None], 
                        'Full Charge Range': [None],'KM before CS': [None],'Required time(If user provide)': [None],'SOC Final': [None],'Required Time (Calculated)': [Y], 'cumulative_time': [None]})

# Insert the new row before the last row
df1 = pd.concat([df1.iloc[:-1], new_row, df1.iloc[-1:]], ignore_index=True)

# Concatenate df1 and df2
final_df = pd.concat([df1, df2], ignore_index=True) 

# Calculate the cumulative sum of required_time and add it as a new column final_cumulative_time
final_df['final_cumulative_time'] = final_df['Required Time (Calculated)'].cumsum()

# Display the resulting dataframe
print("Final DataFrame (with cumulative sum):")
display(final_df)

# Get the last value of cumulative_time
last_cumulative_time = final_df['final_cumulative_time'].iloc[-1]
print(last_cumulative_time)
data_reversed = final_df.iloc[::-1]
if last_cumulative_time > 180 :
    for index, row in data_reversed.iterrows():
        emergency_left_vehicles = emergency_left_vehicles.append(row)
        last_cumulative_time -= row['Required Time (Calculated)']
        final_df.drop(index, inplace=True)
        if last_cumulative_time < 180:
            break
# Display the updated DataFrame and the final value of X
updated_data = final_df.reset_index(drop=True)
emergency_left_vehicles = emergency_left_vehicles.reset_index(drop=True)  
print("Final DataFrame after considering 180 time limit:")      
display(updated_data)  
print("left vehicles data:")   
display(emergency_left_vehicles)   

Final DataFrame (with cumulative sum):


,Car Number,SOC Initial,Preference Slots,Route,Priority,Battery Capacity,Full Charge Range,KM before CS,Required time(If user provide),SOC Final,Required Time (Calculated),cumulative_time,final_cumulative_time
0,23123,35,1,1,0,90.0,350.0,25.0,15.0,30,16,16,16
1,24,35,1,1,0,NaN,NaN,NaN,17.0,31,15,31,31
2,45345,28,1,1,1,NaN,NaN,NaN,15.0,22,13,44,44
3,76786,65,1,1,1,NaN,NaN,NaN,NaN,60,18,62,62
4,5677,28,1,2,2,NaN,NaN,NaN,15.0,24,8,79,70
5,0000,None,None,None,None,None,None,None,None,None,30,None,100
6,5677,28,1,2,2,NaN,NaN,NaN,15.0,24,9,79,109
7,768678,44,1,1,2,NaN,NaN,NaN,NaN,40,14,93,123
8,75675,14,1,2,3,NaN,NaN,NaN,NaN,10,16,109,139
9,452,16,1,3,3,NaN,NaN,NaN,NaN,12,16,125,155


199
Final DataFrame after considering 180 time limit:


C:\Users\USER\AppData\Local\Temp\ipykernel_1012\407445192.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  emergency_left_vehicles = emergency_left_vehicles.append(row)
C:\Users\USER\AppData\Local\Temp\ipykernel_1012\407445192.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  emergency_left_vehicles = emergency_left_vehicles.append(row)


,Car Number,SOC Initial,Preference Slots,Route,Priority,Battery Capacity,Full Charge Range,KM before CS,Required time(If user provide),SOC Final,Required Time (Calculated),cumulative_time,final_cumulative_time
0,23123,35,1,1,0,90.0,350.0,25.0,15.0,30,16,16,16
1,24,35,1,1,0,NaN,NaN,NaN,17.0,31,15,31,31
2,45345,28,1,1,1,NaN,NaN,NaN,15.0,22,13,44,44
3,76786,65,1,1,1,NaN,NaN,NaN,NaN,60,18,62,62
4,5677,28,1,2,2,NaN,NaN,NaN,15.0,24,8,79,70
5,0000,None,None,None,None,None,None,None,None,None,30,None,100
6,5677,28,1,2,2,NaN,NaN,NaN,15.0,24,9,79,109
7,768678,44,1,1,2,NaN,NaN,NaN,NaN,40,14,93,123
8,75675,14,1,2,3,NaN,NaN,NaN,NaN,10,16,109,139
9,452,16,1,3,3,NaN,NaN,NaN,NaN,12,16,125,155


left vehicles data:


,Car Number,SOC Initial,Preference Slots,Route,Priority,Battery Capacity,Full Charge Range,KM before CS,Required time(If user provide),SOC Final,Required Time (Calculated),cumulative_time,final_cumulative_time
0,5645645,36,1,3,3,NaN,NaN,NaN,16.0,32,17,169,199
1,7967,31,1,3,3,NaN,NaN,NaN,NaN,27,13,152,182
